In [ ]:
# Import utils
import numpy as np
import pandas as pd
import copy
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [ ]:
# Setup experiment
experiment_setup = dict(

    # Paths
    path_data = '/home/fesc/dddex/PatientScheduling/Data',
    path_models = '/home/fesc/dddex/PatientScheduling/Data/Models',
    path_results = '/home/fesc/dddex/PatientScheduling/Data/Results',
    
    # Models
    models = dict(
        LSx_LGBM = 'LSx_LGBM',
        LSx_NN_LGBM = 'LSx_NN_LGBM',
        wSAA_RF = 'wSAA_RF',
        SAA_by_area = 'SAA_by_area',
        SAA = 'SAA'
    ),
    
    # Number of scenarios
    K = [10**2, 10**3, 10**4],

    # Time budget multiplier
    rho = [0.85, 1.00, 1.15],

    # n parallel jobs
    n_jobs = 32,
    
    run_suffix = 'xArea_xRoom'
)

# Make all experiment variables visible locally
locals().update(experiment_setup)

# Pre-processing

In [ ]:
# Load and combine all data
results = pd.DataFrame()
for model in models:

    # Read
    result = pd.read_csv(path_results+'/'+model+'_'+run_suffix+'.csv', sep=',')
    result['model'] = copy.deepcopy(model)

    results = pd.concat([results, result])
        
# Finalize
results = results.reset_index(drop=True)

In [ ]:
# Baseline
results = pd.merge(
    left = results.loc[results.model != 'SAA'],
    right = results.loc[results.model == 'SAA', 
                        ['n_scenarios', 'utilization', 'c_waiting_time', 'c_overtime', 'date', 'area', 'room', 'cost']],
    on = ['n_scenarios', 'utilization', 'c_waiting_time', 'c_overtime', 'date', 'area', 'room'],
    suffixes = ('', '_SAA')
)

# Cost performance

In [ ]:
def plot_medians_(grid, x_var, y_var, hue_var, **kwargs):
    
    # Defaults
    linewidth = 1.0
    width=0.66
    fliersize=0
    whis=0

    # Box-plots
    grid.map_dataframe(
        sns.boxplot, x=x_var, y=y_var, hue=hue_var, 
        width=kwargs.get('width', width), fliersize=kwargs.get('fliersize', fliersize), 
        whis=kwargs.get('whis', whis), linewidth=kwargs.get('linewidth', linewidth), 
        palette=kwargs.get('palette', 'magma'))

    # Box-plot sizes
    fac = 0.75

    # iterating through Axes instances
    for ax in grid.axes.flatten():

        # iterating through axes artists:
        for c in ax.get_children():

            # searching for PathPatches
            if isinstance(c, PathPatch):

                # getting current width of box:
                p = c.get_path()
                verts = p.vertices
                verts_sub = verts[:-1]
                xmin = np.min(verts_sub[:, 0])
                xmax = np.max(verts_sub[:, 0])
                xmid = 0.5*(xmin+xmax)
                xhalf = 0.5*(xmax - xmin)

                # setting new width of box
                xmin_new = xmid-fac*xhalf
                xmax_new = xmid+fac*xhalf
                verts_sub[verts_sub[:, 0] == xmin, 0] = xmin_new
                verts_sub[verts_sub[:, 0] == xmax, 0] = xmax_new

                # setting new width of median line
                for l in ax.lines:
                    #if np.all(l.get_xdata() == [xmin, xmax]):
                    if (l.get_xdata()[0] == xmin) & (l.get_xdata()[1] == xmax):
                        l.set_xdata([xmin_new, xmax_new])
                        l.set_linewidth(3.0)
                        l.set_color('red')
                        
                        
    return grid

In [ ]:
def plot_totals_(grid, x_var, y_var, hue_var, **kwargs):
    
    # Point plots
    grid.map_dataframe(
        sns.pointplot, x=x_var, y=y_var, hue=hue_var, 
        dodge=0.5, join=False, markers='x',
        palette=kwargs.get('palette', 'magma'))


    
                        
                        
    return grid

In [ ]:
from matplotlib.patches import PathPatch


def plot_prescriptive_performance(plotData, grid_var, x_var, y_var, hue_var, 
                                  kind='medians', facet_h=6, facet_w=3, **kwargs):
    
    # Defaults
    linewidth = 1.0
    xlabel='overtime penalty'
    ylabel='cost relative to SAA'
    ylim=(-0.05, 1.85)
    
    # Setup facet grid
    grid = sns.FacetGrid(plotData, col=grid_var, height=facet_h, aspect=facet_w/facet_h)
    
    # Box-plots
    if kind=='medians':
        
        grid = plot_medians_(grid, x_var, y_var, hue_var, **kwargs)
        
    elif kind=='totals':
        
        grid = plot_totals_(grid, x_var, y_var, hue_var, **kwargs)
    
    else:
        
        return None
    
    
    # Reference line
    grid.refline(y=1.0, color='red', linewidth=kwargs.get('linewidth', linewidth), linestyle='--')

    # Facet borders
    for ax in grid.axes.flatten(): 
        for _, spine in ax.spines.items():
            spine.set_visible(True) 
            spine.set_color('black')
            spine.set_linewidth(1.0)

    # Axis labels
    for ax in grid.axes.flatten():

        ax.set_xlabel(xlabel=kwargs.get('xlabel', xlabel), fontsize=10, fontweight='bold')
        ax.set_ylabel(ylabel=kwargs.get('ylabel', ylabel), fontsize=10, fontweight='bold')
        ax.title.set_size(10)
        ax.title.set_weight('bold')
        ax.xaxis.set_label_position('bottom')


    # Legend
    grid.add_legend(ncol=1, loc='right')

    # Limits
    grid.set(ylim=kwargs.get('ylim', ylim))

    return grid.figure

In [ ]:
# Select data
K = 10**4
models = ['wSAA_RF', 'LSx_LGBM', 'LSx_NN_LGBM']
min_patients = 2

# Set labels
labels = pd.DataFrame({'model': ['wSAA_RF', 'LSx_LGBM', 'LSx_NN_LGBM'], 
                       'order': [1, 2, 3], 
                       'label': ['wSAA RF', 'LSx LGBM', 'LSx NN LGBM']})

## Cost per room per day

In [ ]:
# Prepare data
plotData = results.loc[
    (results.model.isin(models)) & 
    (results.n_scenarios == K) & 
    (results.n_patients >= min_patients)].copy()

plotData['copres'] = plotData.cost / plotData.cost_SAA
plotData.loc[plotData.cost == plotData.cost_SAA, 'copres'] = 1.0
plotData = plotData.loc[~np.isinf(plotData.copres)].copy()

plotData = pd.merge(
    left=plotData,
    right=labels,
    on='model'
)

plotData = plotData.sort_values('order')

In [ ]:
# Plot
fig = plot_prescriptive_performance(
    plotData=plotData, grid_var='utilization', x_var='c_overtime', y_var='copres', hue_var='label',
    kind='medians', ylim=(-0.05, 1.85), facet_h=6, facet_w=3, fliersize=2, whis=1.5)

In [ ]:
# Save
fig.savefig(path_results+'/Plots/prescriptive_performance_xArea_xRoom_K'+str(K)+'_medians.pdf')  

In [ ]:
# Table
tableData = plotData.groupby(

    [
        'utilization', 
        'c_overtime', 
        'label'
    ]
    
).agg(

    copres = ('copres', np.median)
    
).reset_index()

tableData.pivot(index='label', columns=['utilization', 'c_overtime'], values='copres')

## Total cost

In [ ]:
# Prepare data
plotData = results.loc[
    (results.model.isin(models)) & 
    (results.n_scenarios == K) & 
    (results.n_patients >= min_patients)].copy()

plotData = plotData.groupby(['utilization', 'c_overtime', 'model']).agg(
    cost=('cost', sum),
    cost_SAA=('cost_SAA', sum)
).reset_index()

plotData['copres'] = plotData.cost / plotData.cost_SAA

plotData = pd.merge(
    left=plotData,
    right=labels,
    on='model'
)

plotData = plotData.sort_values('order')

In [ ]:
# Plot
fig = plot_prescriptive_performance(
    plotData=plotData, grid_var='utilization', x_var='c_overtime', y_var='copres', hue_var='label',
    kind='totals', ylim=(-0.05, 1.85), facet_h=6, facet_w=2)

In [ ]:
# Save
fig.savefig(path_results+'/Plots/prescriptive_performance_xArea_xRoom_K'+str(K)+'_totals.pdf')  

In [ ]:
# Table
plotData.pivot(index='label', columns=['utilization', 'c_overtime'], values='copres')

## Stats

In [ ]:
# Significance test

In [ ]:
from scipy import stats

def differences(data, cost_var:str, copres_var:str, groups:list, test='unpaired', alternative='two-sided', n_hypotheses=1, **kwargs):

    """

    ...

    """


    # Initialize
    result = []
        
    # For each group in the data
    for grp, data_ in data.groupby(groups):
        
        models = list(data.model.unique())
        
        # For each model
        for model in models:
        
            # Models to compare to (all others)
            benchmarks = [i for (i, v) in zip(models, [model != m for m in models]) if v]

            # For all models to compare to
            for benchmark in benchmarks:

                # Cost
                cost_model = np.array(data_.loc[data_.model == model][cost_var])
                cost_benchmark = np.array(data_.loc[data_.model == benchmark][cost_var])
            
                # Prescriptive performance
                copres_model_ = np.array(data_.loc[data_.model == model][copres_var])
                copres_benchmark_ = np.array(data_.loc[data_.model == benchmark][copres_var])
                
                # Differences
                with np.errstate(divide='ignore'):

                    diffs_ = (
                        (cost_model == cost_benchmark) * 0 
                        + (cost_model != cost_benchmark) * (copres_model_ - copres_benchmark_)
                    )

                # Remove inf / nan
                diffs = diffs_[np.isfinite(diffs_)]
                copres_model = copres_model_[np.isfinite(copres_model_) & np.isfinite(copres_benchmark_)]
                copres_benchmark = copres_benchmark_[np.isfinite(copres_model_) & np.isfinite(copres_benchmark_)]

                ## Paired test of differences (Wilcoxon Signed Rank Sum Test)
                if test == 'paired':

                    # Mean of differences
                    mean_of_differences = np.mean(diffs)

                    # Median of differences
                    median_of_differences = np.median(diffs)

                    # Share of cases where model is better than benchmark
                    share_model_is_better = sum(diffs < 0) / len(diffs)

                    # Statictical significance
                    statistic, pvalue = stats.wilcoxon(diffs, alternative=alternative, nan_policy='raise')

                    # Store
                    res = {

                        'model': model,
                        'benchmark': benchmark,
                        'mean_of_differences': mean_of_differences,
                        'median_of_differences': median_of_differences,
                        'share_model_is_better': share_model_is_better,
                        'statistic': statistic,
                        'pvalue': pvalue,
                        'sig0001': '***' if pvalue < 0.001 / n_hypotheses else '',
                        'sig0010': '**' if pvalue < 0.01 / n_hypotheses else '',
                        'sig0050': '*' if pvalue < 0.05 / n_hypotheses else ''
                    }

                    # Append
                    result += [res]


                ## Unpaired test of differences (Mann-Whitney U Test)
                elif test == 'unpaired':

                    # Difference of means
                    difference_of_means = np.mean(copres_model) - np.mean(copres_benchmark)

                    # Difference of medians
                    difference_of_medians = np.median(copres_model) - np.median(copres_benchmark)

                    # Share of cases where model is better than benchmark
                    share_model_is_better = sum(diffs < 0) / len(diffs)

                    # Statictical significance
                    statistic, pvalue = stats.mannwhitneyu(copres_model, copres_benchmark, 
                                                           alternative=alternative, nan_policy='raise')

                    # Store
                    res = {

                        **dict(zip(grps, list(grp))),
                    
                        **{
                            'model': model,
                            'benchmark': benchmark,
                            'difference_of_means': difference_of_means,
                            'difference_of_medians': difference_of_medians,
                            'share_model_is_better': share_model_is_better,
                            'statistic': statistic,
                            'pvalue': pvalue,
                            'sig0001': '***' if pvalue < 0.001 / n_hypotheses else '',
                            'sig0010': '**' if pvalue < 0.01 / n_hypotheses else '',
                            'sig0050': '*' if pvalue < 0.05 / n_hypotheses else ''
                        }
                    }

                    # Append
                    result += [res]

    # Result         
    return pd.DataFrame(result)

In [ ]:
# Prepare data
statsData = results.loc[
    (results.model.isin(models)) & 
    (results.n_scenarios == K) & 
    (results.n_patients >= min_patients)].copy()

statsData['copres'] = statsData.cost / statsData.cost_SAA
statsData.loc[statsData.cost == statsData.cost_SAA, 'copres'] = 1.0

In [ ]:
differences(statsData, cost_var='cost', copres_var='copres', groups=['utilization', 'c_overtime'], 
            test='unpaired', alternative='two-sided', n_hypotheses=1)

# <<<<< ARCHIVE >>>>>

In [ ]:
magma = sns.color_palette('magma')
magma

In [ ]:
viridis = sns.color_palette('viridis')
viridis

In [ ]:
palette = {"wSAA RF": magma[2],
           "LSx LGBM": viridis[3], 
           "LSx NN LGBM": magma[5]}

In [ ]:
palette = {"wSAA RF":"tab:cyan",
           "LSx LGBM":"tab:orange", 
           "LSx NN LGBM":"tab:purple"}


In [ ]:
def plotCoPres(data, figsize=(9,4), dotsize=2, display=0.95, **kwargs):

    if len(data.model.unique()) > 5:
        print('Max number of models exceeded (limited to 5 models)')
        return None
    
    # Model colors
    colors = ['blue', 'black', 'red', 'green', 'yellow']
    positions = [0,1,2,3,4]
    
    # Specify limits showing at least 'upper' percent of data
    ylim_top = data.groupby(['c_overtime', 'model']).agg(q = ('pq', lambda x: np.quantile(x, display))).reset_index().q.max()
    ylim_bottom = data.pq.min()*0.66

    # Initialize figure / sub-plots
    fig, axes = plt.subplots(ncols=len(data.c_overtime.unique()), sharey=True)
    fig.subplots_adjust(wspace=0)

    # Create and iterate over sub-plots (per service level)
    for ax, c_overtime in zip(axes, data.c_overtime.unique()):

        # Volins (per model)
        vp = ax.violinplot([data.pq[(data.c_overtime==c_overtime) & (data.model==model)] for model in data.model.unique()], 
                           positions=[positions[i] for i in range(len(data.model.unique()))], showextrema=False)

        # Dots (per model)
        dp = sns.swarmplot(ax=ax, x='model', y='pq', hue='model', data=data.loc[data.c_overtime==c_overtime], 
                           size=dotsize, alpha=0.33, palette=[colors[i] for i in range(len(data.model.unique()))])
        
        # Boxplots (per model)
        bp = ax.boxplot(x=[data.pq[(data.c_overtime==c_overtime) & (data.model==model)] for model in data.model.unique()],
                        positions=[positions[i] for i in range(len(data.model.unique()))], 
                        medianprops={'color': 'black', 'linewidth': 2.5}, 
                        widths=0.33, showfliers=False, showcaps=False, whis=0, patch_artist=True)

        # Baseline (per model)
        ax.axhline(y=1, linewidth=1, linestyle='--', color='grey')

        # Color coding for violins and boxplots
        for i in range(len(data.model.unique())):

            # Violins
            vp['bodies'][i].set_facecolor('grey')

            # Boxplots
            bp['boxes'][i].set_color(colors[i])
            bp['boxes'][i].set_facecolor([0,0,0,0])
            #bp['whiskers'][i].set_color(colors[i])
            #bp['caps'][i].set_color(colors[i])
            bp['medians'][i].set_color(colors[i])


        # Remove x-axis ticks
        ax.tick_params(
            axis='x',          
            which='both',      
            bottom=False,      
            labelbottom=False) 

        # Set axis inner axis titles
        ax.set_xlabel(xlabel='{:.2f}'.format(c_overtime))
        ax.set_ylabel(ylabel='normalized cost', fontsize=10, fontweight='bold')
        ax.xaxis.set_label_position('bottom')
        ax.label_outer()

        # Set limits
        ax.set_ylim(bottom=0.6, top=1.1)

        # Add margin
        ax.margins(0.05) 

        # Remove legends per sub-plot
        ax.get_legend().remove()


    # Add figure legend    
    fig.legend(
        [bp['boxes'][i] for i in range(len(data.model.unique()))], 
        data.model.unique(), 
        bbox_to_anchor=[0.5, 0.95], 
        loc='center', ncol=len(data.model.unique())
    )
    
    

    # Add figure sub-title (below)
    plt.title('Overtime penalty cost', fontsize=10, fontweight='bold', x=-0.5, y=-0.175)

    # Set size
    fig.set_size_inches(figsize)

    # Show plot
    plt.show()

    return fig

In [ ]:
sel = (results.n_scenarios == 10**4) & (results.utilization == 1) & (~np.isinf(results.pq)) & (results.n_patients > 1)
plotData = results.loc[sel]

In [ ]:
plot = plotCoPres(plotData, dotsize=1)

In [ ]:
import matplotlib.gridspec as gridspec


rho = [0.85, 1.00, 1.15]
c_overtime = [1.00, 2.00, 3.00]


fig = plt.figure(figsize=(12, 4))

data = results.loc[(results.n_scenarios == 10**4) & (~np.isinf(results.pq)) & (results.n_patients > 1)].copy()

positions = [0,1,2,3,4]




outer = gridspec.GridSpec(1, len(rho), wspace=0.2, hspace=0.2)

counter = 0

ax_ = {}


for i in range(len(rho)):
    inner = gridspec.GridSpecFromSubplotSpec(1, len(c_overtime),
                    subplot_spec=outer[i], wspace=0, hspace=0.1)
    
    for j in range(len(c_overtime)):
        
        ax = plt.Subplot(fig, inner[j])
        
        # Boxplots (per model)
        
        sel = (data.utilization==rho[i]) & (data.c_overtime==c_overtime[j])
        
        ax.boxplot(
            x=[data.pq[sel & (data.model==model)] for model in data.model.unique()],
            positions=[positions[p] for p in range(len(data.model.unique()))], 
            medianprops={'color': 'black', 'linewidth': 2.5}, 
            widths=0.33, patch_artist=True, showfliers=False)
            #showcaps=False, whis=0)
            
        # Baseline (per model)
        ax.axhline(y=1, linewidth=1, linestyle='--', color='grey')

        
        
        
        
        #t = ax.text(0.5,0.5, 'outer=%d, inner=%d' % (i, j))
        #t.set_ha('center')
        ax.set_xticks([])
        ax.set_yticks([])
        
        if counter > 1:
            
            ax_[counter] = fig.add_subplot(ax, sharey=ax_[counter-1])
            
        else:
            
            ax_[counter]  = fig.add_subplot(ax)
            
            
        counter += 1


plt.setp(ax_[8].get_yticklabels(), visible=False)

plt.show()

In [ ]:
data = results.loc[(results.n_scenarios == 10**4) & (~np.isinf(results.pq)) & (results.n_patients > 1)].copy()



rho = [0.85, 1.00, 1.15]
c_overtime = [1.00, 2.00, 3.00]


colors = ['blue', 'black', 'red', 'green', 'yellow']
positions = [0,1,2,3,4]
    

# Plotting all the subplots
fig, axes = plt.subplots(1, len(rho)*len(c_overtime), sharey=True)
fig.subplots_adjust(wspace=0)

counter = 0
    
for i in range(len(rho)):
    for j in range(len(c_overtime)):
        
        # Data selector
        sel = (data.utilization==rho[i]) & (data.c_overtime==c_overtime[j])
        
        # Boxplots (per model)
        bp = axes[counter].boxplot(
            x=[data.pq[sel & (data.model==model)] for model in data.model.unique()],
            positions=[positions[p] for p in range(len(data.model.unique()))], 
            medianprops={'color': 'black', 'linewidth': 2.5}, 
            widths=0.33, patch_artist=True, showfliers=False,
            showcaps=False, whis=0)
            
        # Baseline (per model)
        bl = axes[counter].axhline(y=1, linewidth=1, linestyle='--', color='grey')        
        
        # Color coding for violins and boxplots
        for m in range(len(data.model.unique())):

            # Boxplots
            bp['boxes'][m].set_color(colors[m])
            bp['boxes'][m].set_facecolor([0,0,0,0])
            #bp['whiskers'][m].set_color(colors[m])
            #bp['caps'][m].set_color(colors[m])
            bp['medians'][m].set_color(colors[m])
            
        # Remove x-axis ticks
        axes[counter].tick_params(
            axis='x',          
            which='both',      
            bottom=False,      
            labelbottom=False) 

        # Set axis and inner axis titles
        axes[counter].set_xlabel(xlabel='{:.2f}'.format(c_overtime[j]))
        axes[counter].set_ylabel(ylabel='Cost relative to cost of SAA', fontsize=10, fontweight='bold')
        axes[counter].xaxis.set_label_position('bottom')
        if j == 1:
            axes[counter].set_title('rho = '+str(rho[i]), loc='center', y=1)
        axes[counter].label_outer()

        # Set limits
        #axes[counter].set_ylim(bottom=0.6, top=1.1)

        # Add margin
        #axes[counter].margins(0.05) 

        # Remove legends per sub-plot
        #axes[counter].get_legend().remove()
            
        # Plot counter
        counter += 1
        
# Add figure legend    
fig.legend(
    [bp['boxes'][m] for m in range(len(data.model.unique()))], 
    data.model.unique(), 
    bbox_to_anchor=[0.5, 0.99], 
    loc='center', ncol=len(data.model.unique())
)



# Add figure sub-title (below)
#plt.title('Overtime penalty cost', fontsize=10, fontweight='bold', x=-3.5, y=-0.175)
#plt.title('Utilization', fontsize=10, fontweight='bold', x=-3.5, y=1)
plt.title('Overtime penalty cost', fontsize=10, fontweight='bold', x=-3.5, y=-0.175)


# Set size
fig.set_size_inches((12,4))

    
#plt.tight_layout()
plt.show()

In [ ]:
# Ideas to try

## report mean
## compare each model with each model directly
## conduct significance test
## report totals
## normalize with SAA by area ==> done, makes no sense
## filter results by area and also by room size

# TBD: could also implement LQRF

In [ ]:
## By area
g = sns.FacetGrid(data, col="utilization", row="area", height=3.5, aspect=.65)
g.map_dataframe(sns.boxplot, x="c_overtime", y="pq", fliersize=0, whis=0,
                hue="model", palette='viridis')
g.set(ylim=(0.55, 1.15))
g.refline(y=1, color='red')

g.add_legend()

plt.show()

In [ ]:
data.groupby('n_patients').agg(n=('cost', lambda x: len(x) / (4*3*3)))

In [ ]:
## By n patients
g = sns.FacetGrid(data, col="utilization", row="n_patients", height=3.5, aspect=.65)
g.map_dataframe(sns.boxplot, x="c_overtime", y="pq", fliersize=0, whis=0,
                hue="model", palette='viridis')
g.set(ylim=(0.55, 1.15))
g.refline(y=1, color='red')

g.add_legend()

plt.show()

In [ ]:
import seaborn as sns
sns.set_theme(style="ticks", palette="pastel")

# Load the example tips dataset
#tips = sns.load_dataset("tips")

# Draw a nested boxplot to show bills by day and time
sns.boxplot(x="c_overtime", y="pq",
            hue="model", 
            data=data)
sns.despine(offset=10, trim=True)

In [ ]:
g = sns.FacetGrid(data, col="utilization", height=3.5, aspect=.65)
g.map(myplot, data)